In [1]:
# 图像拼接思路
'''
1.读图片
2.灰度化
3.计算各自的特征点和描述子
4.匹配特征
5.根据匹配特征计算单应性矩阵
6.对其中一张图片进行透视变换
7创建大图放入两张图片
'''
pass

In [13]:
import cv2
import numpy as np

img1= cv2.imread('../data/img01.jpeg')
# img1 = cv2.resize(img1,None,fx =0.2,fy =0.2)
gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2= cv2.imread('../data/img02.jpeg')
# img2 = cv2.resize(img2,None,fx =0.2,fy =0.2)
gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
print(gray1.shape,gray2.shape) # 最好两张图像弄成一样大
# 检测特征
sift = cv2.xfeatures2d.SIFT_create() #SIFT_create([, nfeatures[, nOctaveLayers[, contrastThreshold[, edgeThreshold[, sigma]]]]]) -> retval

kp1,des1 = sift.detectAndCompute(gray1,None)
kp2,des2 = sift.detectAndCompute(gray2,None)

bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2,k=2)

# 筛选匹配对
good_matches = []
for m, n in matches:
    if m.distance < 0.7* n.distance:
        good_matches.append(m)
        
# out_img = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None)
# cv2.imwrite('feature_pairing_diagram.jpg',out_img)
if len(good_matches)>=4:
    src_point = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2) # 这里要注意关系
    dst_point = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)
    # 计算单应性矩阵
    H, _ =cv2.findHomography(src_point,dst_point,cv2.RANSAC,5) # 注意这个变换视角是左图对右图的
else:
    print('没有足够的特征点')

    
# 开始进行变换
h1,w1 = img1.shape[:2]
h2,w2 = img2.shape[:2]

img1_pts = np.float32([[0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0]]).reshape(-1,1,2) # 获取原始图的四个边界点
img2_pts = np.float32([[0,0],[0,h2-1],[w2-1,h2-1],[w2-1,0]]).reshape(-1,1,2)

# 根据H矩阵变换img1的四个角点
img1_transform_pts = cv2.perspectiveTransform(img1_pts,H)
img1_transform_pts

(2128, 4608) (2128, 4608)


array([[[-3450.3704  ,  -778.14355 ]],

       [[-3234.42    ,  2581.506   ]],

       [[ 2843.5527  ,  1780.1897  ]],

       [[ 2840.185   ,   -20.043798]]], dtype=float32)

![img](feature_pairing_diagram.jpg)

In [21]:
aaa = cv2.warpPerspective(img1,H,(4608,2128))
aaa =cv2.resize(aaa,(0,0),fx=0.1,fy=0.1)
cv2.imshow('ex',aaa)
cv2.waitKey(0)
cv2.destroyAllWindows()
# 尝试观察一下变换后的图像
# 结合上面的输出,可以知道,img1变换完之后,画面中很多区域在显示窗口之外了
# 所以要把显示在画面窗口之外的部分平移回来

![image.png](010.png)

In [35]:
result_pts = np.concatenate((img2_pts,img1_transform_pts),axis =0) # 由于要拼接整个图,所以img2(右图也进行平移)
print(result_pts)

# 获取要平移的值
[[x_min,y_min]] = np.int32(result_pts.min(axis=0,)-1) # 平移值需要是整数 (-1 是确保完全平移值能大于负数的绝对值)
print(x_min,y_min)

[[[    0.           0.      ]]

 [[    0.        2127.      ]]

 [[ 4607.        2127.      ]]

 [[ 4607.           0.      ]]

 [[-3450.3704    -778.14355 ]]

 [[-3234.42      2581.506   ]]

 [[ 2843.5527    1780.1897  ]]

 [[ 2840.185      -20.043798]]]
-3451 -779


In [37]:
# 获取拼接之后的图需要多大
[[x_max,y_max]] = np.int32(result_pts.max(axis=0,)+1)
print(x_max,y_max)

4608 2582


In [52]:
# 构造平移矩阵
move_matrix = np.array([[1,0,-x_min],[0,1,-y_min],[0,0,1]])

# 对img1进行平移变换及透视变换
M = move_matrix.dot(H) # 先平移再透视变换
# M = H.dot(move_matrix)
res_img = cv2.warpPerspective(img1,M,(x_max-x_min,y_max-y_min)) 
# (x_max-x_min,y_max-y_min) 这里是求最后的窗口应该是多大
bbb =cv2.resize(res_img,(0,0),fx=0.1,fy=0.1)
cv2.imshow('ex',bbb)
cv2.waitKey(0)
cv2.destroyAllWindows()


![image.png](011.png)

In [61]:
# 把img2放进来
res_img[-y_min:-y_min+h2,-x_min:-x_min+w2] = img2

bbb =cv2.resize(res_img,(0,0),fx=0.2,fy=0.2)
cv2.imshow('ex',bbb)
cv2.waitKey(0)
cv2.destroyAllWindows()


![image.png](012.png)